In [1]:
import dolfinx # FEM in python
import matplotlib.pyplot as plt
import ufl # variational formulations
import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import gmsh

In [2]:
import pyvista # visualisation in python notebook
# pyvista.set_jupyter_backend("pythreejs") # interactif, mais pas super
pyvista.set_jupyter_backend("none") # non-interactif, mais mieux
# NB : ce truc peut se changer a chaque plot

In [3]:
import extract # this to be able to have the value of a solution at point (x,y)
# use it this way : extract.solution(my_domain, u_solution, x=0.5, y=0.01)

In [4]:
R_i = 1.0 # Radius of the inclusion
R_e = 6.9  # Radius of the matrix (whole domain)
aspect_ratio = 1.0 #0.75
mesh_size = 0.1*R_i
mesh_order = 1 

mesh_comm = MPI.COMM_WORLD
model_rank = 0
gmsh.initialize()
facet_names = {"inner_boundary": 1, "outer_boundary": 2}
cell_names = {"inclusion": 1, "matrix": 2}
model = gmsh.model()
model.add("Disk")
model.setCurrent("Disk")
gdim = 2 # geometric dimension of the mesh
inner_disk = gmsh.model.occ.addDisk(0, 0, 0, R_i, aspect_ratio * R_i)
outer_disk = gmsh.model.occ.addDisk(0, 0, 0, R_e, R_e)
whole_domain = gmsh.model.occ.fragment(
            [(gdim, outer_disk)], [(gdim, inner_disk)]
        )
gmsh.model.occ.synchronize()
# Add physical tag for bulk
inner_domain = whole_domain[0][0]
outer_domain = whole_domain[0][1]
model.addPhysicalGroup(gdim, [inner_domain[1]], tag=cell_names["inclusion"])
model.setPhysicalName(gdim, inner_domain[1], "Inclusion")
model.addPhysicalGroup(gdim, [outer_domain[1]], tag=cell_names["matrix"])
model.setPhysicalName(gdim, outer_domain[1], "Matrix")

# Add physical tag for boundaries
lines = gmsh.model.getEntities(dim=1)
inner_boundary = lines[1][1]
outer_boundary = lines[0][1]
gmsh.model.addPhysicalGroup(1, [inner_boundary], facet_names["inner_boundary"])
gmsh.model.addPhysicalGroup(1, [outer_boundary], facet_names["outer_boundary"])
gmsh.option.setNumber("Mesh.CharacteristicLengthMin",mesh_size)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax",mesh_size)
model.mesh.generate(gdim)
gmsh.option.setNumber("General.Terminal", 1)
model.mesh.setOrder(mesh_order)
gmsh.option.setNumber("General.Terminal", 0)

# Import the mesh in dolfinx
from dolfinx.io import gmshio
domain, cell_tags, facet_tags = gmshio.model_to_mesh(model, mesh_comm, model_rank, gdim=gdim)
domain.name = "composite"
cell_tags.name = f"{domain.name}_cells"
facet_tags.name = f"{domain.name}_facets"
gmsh.finalize()

# Save the mesh in XDMF format
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "output/mesh.xdmf", "w") as file:
    file.write_mesh(domain)
    domain.topology.create_connectivity(1, 2)
    file.write_meshtags(cell_tags, geometry_xpath=f"/Xdmf/Domain/Grid[@Name='{domain.name}']/Geometry")
    file.write_meshtags(facet_tags, geometry_xpath=f"/Xdmf/Domain/Grid[@Name='{domain.name}']/Geometry")


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.0016088s, CPU 0.00315s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.884689s, CPU 0.835872s)
Info    : 17662 nodes 35387 elements


In [5]:
ds = ufl.Measure("ds", subdomain_data=facet_tags, domain=domain)
dx = ufl.Measure("dx", subdomain_data=cell_tags, domain=domain)
one = dolfinx.fem.Constant(domain,ScalarType(1.))
surface = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * dx))
surface_inner = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * dx(cell_names["inclusion"])))
surface_outer = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * dx(cell_names["matrix"])))
print(f"Total surface: {surface:10.4f}, Inner surface: {surface_inner:10.4f}, Outer surface: {surface_outer:10.4f}")

Total surface:   149.5660, Inner surface:     3.1364, Outer surface:   146.4296


In [6]:
V = dolfinx.fem.VectorFunctionSpace(domain,("Lagrange", 1),dim=2)
def eps(u):
    return ufl.sym(ufl.grad(u))
E_m = 0.8 # Young's modulus in matrix
nu_m = 0.35 # Poisson's ratio in matrix
E_i = 11.0
nu_i = 0.3

I2 = ufl.Identity(2)
#mu = dolfinx.fem.Constant(domain, ScalarType( E/2./(1.+nu) ))
#lamb = E*nu/(1.+nu)/(1.-2.*nu)

# Hook's law is written as the top of this notebook
def sigma(eps, E, nu):
    mu = E/2./(1.+nu)
    lamb = E*nu/(1.+nu)/(1.-2.*nu)
    return lamb*ufl.tr(eps)*I2 + 2*mu*eps

u = ufl.TrialFunction(V)
u_bar = ufl.TestFunction(V)

bilinear_form_inclusion = ( ufl.inner(sigma(eps(u),E_i,nu_i),eps(u_bar)) ) * dx(1)
bilinear_form_matrix = ( ufl.inner(sigma(eps(u),E_m,nu_m),eps(u_bar)) ) * dx(2)
bilinear_form = bilinear_form_inclusion + bilinear_form_matrix
g=0.0 # some weight
body_force = dolfinx.fem.Constant(domain, ScalarType((0,-g)))
linear_form = ( ufl.dot(body_force,u_bar)  ) * ufl.dx

In [7]:
outer_facets = facet_tags.find(facet_names["outer_boundary"])

outer_boundary_dofs = dolfinx.fem.locate_dofs_topological(V, gdim-1,outer_facets)


In [8]:
uD = dolfinx.fem.Function(V)

u_on_boundary = lambda x: np.array([-x[1], -x[0]], dtype=ScalarType)

uD.interpolate(u_on_boundary)

bc = dolfinx.fem.dirichletbc(uD, outer_boundary_dofs)

In [9]:
problem = dolfinx.fem.petsc.LinearProblem(bilinear_form, linear_form, bcs=[bc], 
                                          petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
u_solution = problem.solve()

In [10]:
np.sqrt(dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.dot(u_solution,u_solution) * ufl.dx)))

59.0355945143035